# Conversor BioC XML -> TXT CONLL

Fontes importantes:
- Doc CSV: https://docs.python.org/3/library/csv.html
- Doc XML Element Tree: https://docs.python.org/3/library/xml.etree.elementtree.html

## Conversor BioC XML -> CSV

### Append de arquivos xml e instanciação de lista de roots

In [ ]:
import xml.etree.ElementTree as ET

# lista de xmls para criação de roots
xmls = []

xml1 = './xmls/dodf1_p2.xml'
xmls.append(xml1)
xml2 = './xmls/dodf1_p3.xml'
xmls.append(xml2)
xml3 = './xmls/dodf1_p4.xml'
xmls.append(xml3)
xml4 = './xmls/dodf2_p1.xml'
xmls.append(xml4)
xml5 = './xmls/dodf2_p2.xml'
xmls.append(xml5)
xml6 = './xmls/dodf2_p3.xml'
xmls.append(xml6)
xml7 = './xmls/dodf2_p4.xml'
xmls.append(xml7)

# lista de roots para iteração posterior
roots = []

# populando lista de roots iterando na lista xmls
for xml in xmls:
    tree = ET.parse(xml)
    root = tree.getroot()
    roots.append(root)

### Criação arquivo CSV

Iteramos sobre as roots de cada XML. Encontramos todas as entidades 'annotation', em seguida guardamos os id's de cada uma. Depois iteramos na lista de id's e identificamos o tipo e o texto de cada objeto 'annotation', e salvamos isso, respectivamente, em um CSV de colunas 'tipo_ato' e 'ato'

In [ ]:
import csv

# abre csv para escrita de headers
with open('atos_segs.csv', 'a') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(['tipo_ato', 'ato'])

# abre csv para escrita em modo append
with open('atos_segs.csv', 'a') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    
    # itera sobre lista de roots dos xmls
    for root in roots:
        
        # cria lista de ids
        for anno in root.findall("./document/passage/annotation"):
            ids = []
            id = anno.get('id')
            ids.append(id)

            # loop na lista de ids
            for i in ids:
                # encontra e itera sobre todos os elementos annotation do xml
                for anno in root.findall("./document/passage/annotation"):
                    # para cada anotação definida por um id, pegue o tipo e o texto do ato
                    if anno.get('id') == i:
                        for info in anno.findall('infon'):
                            if info.get('key') == 'type':
                                tipoAnno = info.text
                                # encontra primeiro elemento filho de 'info' com tag
                                # 'text' e salva o texto em variavel
                                texto = anno.find('text').text
                                # escreve linha (tipo_ato, ato) no csv
                                writer.writerow([tipoAnno, texto])

## CSV -> TXT CONLL

### Import em arquivo csv gerado

In [1]:
import pandas as pd

dt = pd.read_csv('atos_segs.csv')
dt

,tipo_ato,ato
0,nomeacao,NOMEAR LETÍCIA MATOS MAGALHÃES para exercer o ...
1,exoneracao,"EXONERAR, a pedido, ANNA LUIZA BARCIA HALLEY d..."
2,nomeacao,NOMEAR SANDRA TURCATO JORGE TOLENTINO para exe...
3,nomeacao,"NOMEAR RAIMUNDO DA COSTA SANTOS NETO, Procurad..."
4,ato_tornado_sem_efeito,TORNAR SEM EFEITO no Decreto de 02 de julho de...
...,...,...
268,substituicao,DESIGNAR\nrespectivamente TEREZA CRISTINA DE A...
269,substituicao,"Designar FLÁVIO DE ARAÚJO ALMEIDA, matrícula n..."
270,substituicao,"DESIGNAR OTAMÁ DANTAS BARRETO, matrícula\n159...."
271,retificacao,"RETIFICAR o Sexto Termo\nAditivo ao Contrato, ..."


In [2]:
dt.tipo_ato.value_counts()

aposentadoria             69
nomeacao                  53
exoneracao                49
retificacao               38
substituicao              37
ato_tornado_sem_efeito    16
cessao                     9
reversao                   1
abono_de_permanência       1
Name: tipo_ato, dtype: int64

### Retira classes com apenas 1 instância

In [3]:
dt = dt[dt.tipo_ato != 'abono_de_permanência']
dt = dt[dt.tipo_ato != 'reversao']
dt.tipo_ato.value_counts()

aposentadoria             69
nomeacao                  53
exoneracao                49
retificacao               38
substituicao              37
ato_tornado_sem_efeito    16
cessao                     9
Name: tipo_ato, dtype: int64

### Cria listas de atos e tipos de ato, em ordem

In [4]:
col_names = ['tipo_ato', 'ato']
df = pd.read_csv('atos_segs.csv', names=col_names)
atos = df.ato.to_list()
tipos = df.tipo_ato.to_list()
atos.remove('ato')
tipos.remove('tipo_ato')

### Retira \n das instâncias de atos

In [5]:
import re

at_contiguos = []
for ato in atos:
    at_contiguos.append(re.sub('\n', ' ', ato))
at_contiguos

['NOMEAR LETÍCIA MATOS MAGALHÃES para exercer o Cargo em Comissão, Símbolo CC-04, código SIGRH B0001615, de Assessor Técnico, da Chefia de Gabinete, do Gabinete do Governador.',
 'EXONERAR, a pedido, ANNA LUIZA BARCIA HALLEY do Cargo de Natureza Especial, Símbolo CNE-04, código SIGRH 00701906, de Assessor Especial, da Assessoria de Comunicação, do Gabinete, da Secretaria de Estado de Economia do Distrito Federal.',
 'NOMEAR SANDRA TURCATO JORGE TOLENTINO para exercer o Cargo de Natureza Especial, Símbolo CNE-04, código SIGRH 00701906, de Assessor Especial, da Assessoria de Comunicação, do Gabinete, da Secretaria de Estado de Economia do Distrito Federal.',
 'NOMEAR RAIMUNDO DA COSTA SANTOS NETO, Procurador do Distrito Federal, matrícula 119221-3, para exercer Cargo Público de Natureza Especial, Símbolo CPE-03, código SIGRH 65260043, de Chefe, da Assessoria Jurídico-Legislativa, da Secretaria de Estado de Educação do Distrito Federal.',
 'TORNAR SEM EFEITO no Decreto de 02 de julho de 2

### Cria lista de listas de referência

infos é uma lista que contém duas outras listas:
- tipos
- at_contiguos

Ela servirá de referência para mapear instâncias

In [6]:
infos = []
infos.append(tipos)
infos.append(at_contiguos)
infos

infos[0][1]

'exoneracao'

### Tokeniza entradas de atos contíguos

Criamos uma lista anteriormente chamada at_contiguos, que contém os textos de atos sem \n's. Esse bloco de código a seguir serve para dividir em tokens esses textos.

In [7]:
import nltk

at_tokenizados = []

for ato in at_contiguos:
    nltk_tokens = nltk.word_tokenize(ato)
    at_tokenizados.append(nltk_tokens)
    
at_tokenizados[192]

['o',
 'ato',
 'que',
 'exonerou',
 ',',
 'por',
 'estar',
 'sendo',
 'nomeado',
 'para',
 'outro',
 'cargo',
 'e',
 'nomeou',
 'o',
 '2º',
 'SGT',
 'QBMG-1',
 'MARCELO',
 'OLIVIERA',
 'BATISTA',
 ',',
 'ONDE',
 'SE',
 'LÊ',
 ':',
 '“',
 '...',
 'MARCELO',
 'OLIVIERA',
 'BATISTA',
 '...',
 '”',
 ',',
 'LEIASE',
 ':',
 '“',
 '...',
 'MARCELO',
 'OLIVEIRA',
 'BATISTA',
 '...',
 '”',
 '.']

## Dividir entrada em treino e teste

### Mapeia labels

In [8]:
possible_labels = dt.tipo_ato.unique()
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [9]:
dt['label'] = dt.tipo_ato.replace(label_dict)
dt

,tipo_ato,ato,label
0,nomeacao,NOMEAR LETÍCIA MATOS MAGALHÃES para exercer o ...,0
1,exoneracao,"EXONERAR, a pedido, ANNA LUIZA BARCIA HALLEY d...",1
2,nomeacao,NOMEAR SANDRA TURCATO JORGE TOLENTINO para exe...,0
3,nomeacao,"NOMEAR RAIMUNDO DA COSTA SANTOS NETO, Procurad...",0
4,ato_tornado_sem_efeito,TORNAR SEM EFEITO no Decreto de 02 de julho de...,2
...,...,...,...
268,substituicao,DESIGNAR\nrespectivamente TEREZA CRISTINA DE A...,4
269,substituicao,"Designar FLÁVIO DE ARAÚJO ALMEIDA, matrícula n...",4
270,substituicao,"DESIGNAR OTAMÁ DANTAS BARRETO, matrícula\n159....",4
271,retificacao,"RETIFICAR o Sexto Termo\nAditivo ao Contrato, ...",3


### Abordagem estratificada, dado desbalanceamento de classes

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dt.index.values, 
                                                  dt.label.values, 
                                                  test_size=0.25, 
                                                  random_state=17, 
                                                  stratify=dt.label.values)

In [11]:
X_train, X_test, y_train, y_test

(array([  4, 192,  70, 132,  83, 186,  91, 195,  82,  48, 256,  87,  26,
         69, 136, 204,  22, 179,  27, 216,  72, 261,   0,  80, 169, 157,
        246, 189, 207, 110,  57, 240, 137,  81, 102, 115, 241, 103,  10,
         19, 190,  29,  16, 234, 184, 205, 151, 180,  71,  86, 231,   1,
        257,  45, 174, 188, 260, 218, 214, 202, 248,  21, 139, 173,   5,
        165,  32,  58, 167, 177,  76,  28,   6,  49, 220, 134,  56,   7,
        131, 187, 237, 166,  17, 101, 225, 228, 269, 119,  94, 142, 250,
        217, 107,  41, 252,  78,  55, 135, 270,  59, 201, 211,  34, 153,
         13,  25, 209,  12, 244,  23,  73,  60, 268,  30,  18,  36,  42,
        224, 196, 128,  92, 242, 112,  66, 213,  84, 122, 170,  50, 208,
         43,  46,  53, 222,  37,  24,  74,  54, 105, 109, 159, 100, 198,
        164, 148, 254, 121, 194,  61, 243, 124, 158, 138, 182, 266, 253,
        221, 162, 152, 230, 263, 133,  38,  97,  88,  65, 147,  77,   8,
        145, 108, 212, 168, 203, 183, 239,   3, 226

In [12]:
at_tokenizados = []

for ato in at_contiguos:
    nltk_tokens = nltk.word_tokenize(ato)
    at_tokenizados.append(nltk_tokens)

## Juntando tudo

O bloco de código abaixo ajusta para o formato Conll as entradas de texto já tokenizado, referente às listas de treino (X_train) e validação (X_val). Como a tarefa fim aqui é segmentação, ou detecção de fronteiras entre sentenças (sentence boundary detection), o token inicial de cada ato é marcado com -S + [tipo de ato] e o token final com -E. Os demais tokens tem -O.

In [13]:
import numpy as np

dados_treino = []

# itera sobre array de treino
for valor in np.nditer(X_train):
    # instancia variavel para mapear instancia na lista at_tokenizados
    index_ato = int(valor)
    print(index_ato)
    # instancia variavel para iterar sobre a lista de palavras
    index_token = 0
    ato = at_tokenizados[index_ato]
    print(ato)
    # itera sobre lista de palavras
    for palavra in ato:
        print(palavra, index_token)
        # é o token inicial?
        if index_token == 0:
            palavra = palavra + ' S-'+ infos[0][index_ato]
            ato[index_token] = palavra
        # é o segundo token²
        elif index_token == 1:
            palavra = palavra + ' I-'+ infos[0][index_ato]
            ato[index_token] = palavra
        # é o token final?
        elif index_token == len(at_tokenizados[index_ato])-1:
            # Já temos a informação de classe no token de início.
            palavra = palavra + ' E'
            ato[index_token] = palavra
        else:
            palavra = palavra + ' O'
            ato[index_token] = palavra
        index_token = index_token + 1
    # salva lista de palavras na lista de treino ou teste
    dados_treino.append(ato)

4
['TORNAR', 'SEM', 'EFEITO', 'no', 'Decreto', 'de', '02', 'de', 'julho', 'de', '2020', ',', 'publicado', 'no', 'DODF', 'nº', '124', ',', 'de', '03', 'de', 'julho', 'de', '2020', ',', 'página', '18', ',', 'o', 'ato', 'que', 'nomeou', 'ERNANY', 'SANTOS', 'DE', 'ALMEIDA', 'para', 'exercer', 'o', 'Cargo', 'Público', 'de', 'Natureza', 'Especial', ',', 'Símbolo', 'CPE-02', ',', 'código', 'SIGRH', '65260137', ',', 'de', 'Subsecretário', ',', 'da', 'Subsecretaria', 'de', 'Planejamento', ',', 'Acompanhamento', 'e', 'Avaliação', ',', 'da', 'Secretaria', 'de', 'Estado', 'de', 'Educação', 'do', 'Distrito', 'Federal', '.']
TORNAR 0
SEM 1
EFEITO 2
no 3
Decreto 4
de 5
02 6
de 7
julho 8
de 9
2020 10
, 11
publicado 12
no 13
DODF 14
nº 15
124 16
, 17
de 18
03 19
de 20
julho 21
de 22
2020 23
, 24
página 25
18 26
, 27
o 28
ato 29
que 30
nomeou 31
ERNANY 32
SANTOS 33
DE 34
ALMEIDA 35
para 36
exercer 37
o 38
Cargo 39
Público 40
de 41
Natureza 42
Especial 43
, 44
Símbolo 45
CPE-02 46
, 47
código 48
SIGRH 49

NOMEAR 0
CAMILA 1
GOMES 2
CARVALHO 3
TEIXEIRA 4
, 5
matrícula 6
276.270-6 7
, 8
para 9
exercer 10
o 11
Cargo 12
Público 13
em 14
Comissão 15
, 16
Símbolo 17
CPC-08 18
, 19
SIGRH 20
03100851 21
, 22
de 23
Assessor 24
, 25
da 26
Subcontroladoria 27
de 28
Correição 29
Administrativa 30
, 31
da 32
Controladoria-Geral 33
do 34
Distrito 35
Federal 36
. 37
91
['CONCEDER', 'APOSENTADORIA', 'a', 'EMILIA', 'SOUSA', 'COSTA', 'DO', 'NASCIMENTO', ',', 'matrícula', '26.677-9', ',', 'no', 'Cargo', 'de', 'Professor', 'de', 'Educação', 'Básica', ',', 'Padrão', '25', ',', 'Etapa', 'IV', ',', 'do', 'Quadro', 'de', 'Pessoal', 'do', 'Distrito', 'Federal', ',', 'nos', 'termos', 'do', 'artigo', '3º', ',', 'incisos', 'I', ',', 'II', 'e', 'III', ',', 'e', 'parágrafo', 'único', 'da', 'Emenda', 'Constitucional', 'n°', '47', 'de', '06', 'de', 'julho', 'de', '2005', '.', 'Processo', '0008000211950/2019-98', '.']
CONCEDER 0
APOSENTADORIA 1
a 2
EMILIA 3
SOUSA 4
COSTA 5
DO 6
NASCIMENTO 7
, 8
matrícula 9
26.677-9 10
,

9.144/2017 187
. 188
V 189
- 190
Publique-se 191
e 192
encaminhe-se 193
à 194
Polícia 195
Civil 196
do 197
Distrito 198
Federal 199
, 200
para 201
as 202
providências 203
pertinentes 204
. 205
136
['DESIGNAR', 'LUIZ', 'FELIPE', 'BLANCO', 'DE', 'ALENCAR', ',', 'matrícula', 'nº', '195.158-0', ',', 'Analista', 'de', 'Atividades', 'do', 'Meio', 'Ambiente', ',', 'para', 'substituir', 'MARCUS', 'VINICIUS', 'FALCÃO', 'PAREDES', ',', 'matrícula', '263.877-0', ',', 'Chefe', ',', 'Símbolo', 'CPE-07', ',', 'da', 'Unidade', 'de', 'Educação', 'Ambiental', ',', 'da', 'Secretaria', 'Geral', ',', 'no', 'período', 'de', '20', 'de', 'julho', 'a', '03', 'de', 'agosto', 'de', '2020', ',', 'por', 'motivo', 'de', 'férias', 'regulamentares', 'do', 'titular', ',', 'processo', '0039100005905/2019-00', '.']
DESIGNAR 0
LUIZ 1
FELIPE 2
BLANCO 3
DE 4
ALENCAR 5
, 6
matrícula 7
nº 8
195.158-0 9
, 10
Analista 11
de 12
Atividades 13
do 14
Meio 15
Ambiente 16
, 17
para 18
substituir 19
MARCUS 20
VINICIUS 21
FALCÃO 22
P

da 67
Emenda 68
Constitucional 69
nº 70
20 71
, 72
de 73
16 74
de 75
dezembro 76
de 77
1998 78
, 79
e 80
artigo 81
2º 82
da 83
Emenda 84
Constitucional 85
nº 86
47 87
, 88
de 89
06 90
de 91
julho 92
de 93
2005 94
. 95
Processo 96
00080-00226082/2019-41 97
. 98
169
['EXONERAR', 'LUISA', 'SILVA', 'DUTRA', ',', 'matrícula', '1662207-3', ',', 'do', 'Cargo', 'Público', 'em', 'Comissão', ',', 'Símbolo', 'CPC-03', ',', 'SIGRH', '55005528', ',', 'de', 'Chefe', ',', 'do', 'Núcleo', 'de', 'Admissão', 'e', 'Movimentação', ',', 'da', 'Gerência', 'de', 'Administração', 'de', 'Profissionais', ',', 'da', 'Diretoria', 'de', 'Administração', 'de', 'Profissionais', ',', 'da', 'Coordenação', 'de', 'Administração', 'de', 'Profissionais', ',', 'da', 'Subsecretaria', 'de', 'Gestão', 'de', 'Pessoas', ',', 'da', 'Secretaria', 'de', 'Estado', 'de', 'Saúde', 'do', 'Distrito', 'Federal', '.']
EXONERAR 0
LUISA 1
SILVA 2
DUTRA 3
, 4
matrícula 5
1662207-3 6
, 7
do 8
Cargo 9
Público 10
em 11
Comissão 12
, 13
Símbolo

a 2
ROSA 3
LIMA 4
DE 5
OLIVEIRA 6
, 7
matrícula 8
51.200-1 9
, 10
no 11
Cargo 12
de 13
Técnico 14
de 15
Gestão 16
Educacional/ 17
Educação 18
em 19
Saúde 20
, 21
Nível 22
11 23
, 24
Padrão 25
1 26
, 27
Etapa 28
V 29
, 30
do 31
Quadro 32
de 33
Pessoal 34
do 35
Distrito 36
Federal 37
, 38
nos 39
termos 40
do 41
artigo 42
3º 43
, 44
incisos 45
I 46
, 47
II 48
e 49
III 50
, 51
e 52
parágrafo 53
único 54
da 55
Emenda 56
Constitucional 57
n° 58
47 59
de 60
06 61
de 62
julho 63
de 64
2005 65
. 66
Processo 67
00080-00034065/202012 68
. 69
241
['CONCEDER', 'APOSENTADORIA', 'a', 'ROSILENE', 'GOMES', 'LISBOA', ',', 'matrícula', '41.286-4', ',', 'no', 'Cargo', 'de', 'Agente', 'de', 'Gestão', 'Educacional', ',', 'Nível', '10', 'Padrão', '3', ',', 'Etapa', 'III', ',', 'do', 'Quadro', 'de', 'Pessoal', 'do', 'Distrito', 'Federal', ',', 'nos', 'termos', 'do', 'artigo', '3º', ',', 'incisos', 'I', ',', 'II', 'e', 'III', ',', 'e', 'parágrafo', 'único', 'da', 'Emenda', 'Constitucional', 'n°', '47', 'de', '

...

Internação 225
de 226
Saída 227
Sistemática 228
, 229
da 230
Coordenação 231
de 232
Internação 233
, 234
da 235
Subsecretaria 236
do 237
Sistema 238
Socioeducativo 239
, 240
da 241
Secretaria 242
de 243
Estado 244
de 245
Justiça 246
e 247
Cidadania 248
do 249
Distrito 250
Federal 251
, 252
em 253
seus 254
impedimentos 255
ou 256
afastamentos 257
legais 258
, 259
a 260
contar 261
de 262
31/07/2020 263
. 264
133
['DESIGNAR', 'JULIANE', 'ROSA', 'DE', 'OLIVEIRA', ',', 'matrícula', 'nº', '197.859-4', ',', 'Técnico', 'de', 'Atividades', 'do', 'Meio', 'Ambiente', ',', 'para', 'substituir', 'LUCIANA', 'PEREIRA', 'FERNANDES', ',', 'matrícula', 'nº', '263.916-5', ',', 'Gerente', ',', 'Símbolo', 'CPC-08', ',', 'da', 'Gerência', 'de', 'Compras', 'e', 'Contratos', ',', 'da', 'Diretoria', 'de', 'Logística', 'e', 'Infraestrutura', ',', 'da', 'Superintendência', 'de', 'Administração', 'Geral', ',', 'no', 'período', 'de', '03', 'a', '22', 'de', 'agosto', 'de', '2020', ',', 'por', 'motivo', 'de', 'féria

17/07/2010 48
a 49
16/07/2015 50
, 51
LEIA-SE 52
: 53
“ 54
17/07/2010 55
a 56
15/07/2015 57
” 58
, 59
168
['NOMEAR', 'SEBASTIANA', 'COELHO', 'COSTA', 'CONDE', ',', 'Técnico', 'Administrativo', ',', 'matrícula', '0188981-8', ',', 'para', 'exercer', 'o', 'Cargo', 'Público', 'em', 'Comissão', ',', 'Símbolo', 'CPC-03', ',', 'SIGRH', '55005531', ',', 'de', 'Chefe', ',', 'do', 'Núcleo', 'de', 'Cessões', 'Especiais', ',', 'da', 'Gerência', 'de', 'Profissionais', 'Cedidos', 'e', 'Requisitados', ',', 'da', 'Diretoria', 'de', 'Administração', 'de', 'Profissionais', ',', 'da', 'Coordenação', 'de', 'Administração', 'de', 'Profissionais', ',', 'da', 'Subsecretaria', 'de', 'Gestão', 'de', 'Pessoas', ',', 'da', 'Secretaria', 'de', 'Estado', 'de', 'Saúde', 'do', 'Distrito', 'Federal', '.']
NOMEAR 0
SEBASTIANA 1
COELHO 2
COSTA 3
CONDE 4
, 5
Técnico 6
Administrativo 7
, 8
matrícula 9
0188981-8 10
, 11
para 12
exercer 13
o 14
Cargo 15
Público 16
em 17
Comissão 18
, 19
Símbolo 20
CPC-03 21
, 22
SIGRH 23
5

Cargo 12
de 13
Agente 14
de 15
Gestão 16
Educacional/Portaria 17
, 18
Nível 19
11 20
, 21
Padrão 22
1 23
, 24
Etapa 25
IV 26
, 27
do 28
Quadro 29
de 30
Pessoal 31
do 32
Distrito 33
Federal 34
, 35
nos 36
termos 37
do 38
artigo 39
3º 40
, 41
incisos 42
I 43
, 44
II 45
e 46
III 47
, 48
e 49
parágrafo 50
único 51
da 52
Emenda 53
Constitucional 54
n° 55
47 56
de 57
06 58
de 59
julho 60
de 61
2005 62
. 63
Processo 64
0008000003933/2020-12 65
. 66
52
['NOMEAR', 'BIANCA', 'SOUZA', 'DA', 'SILVA', 'para', 'exercer', 'Cargo', 'em', 'Comissão', ',', 'Símbolo', 'CC02', ',', 'de', 'Assessor', 'Técnico', ',', 'da', 'Coordenação', 'de', 'Licenciamento', ',', 'Obras', 'e', 'Manutenção', ',', 'da', 'Administração', 'Regional', 'do', 'Plano', 'Piloto', 'do', 'Distrito', 'Federal', '.']
NOMEAR 0
BIANCA 1
SOUZA 2
DA 3
SILVA 4
para 5
exercer 6
Cargo 7
em 8
Comissão 9
, 10
Símbolo 11
CC02 12
, 13
de 14
Assessor 15
Técnico 16
, 17
da 18
Coordenação 19
de 20
Licenciamento 21
, 22
Obras 23
e 24
Manutenção 25
,

, 16
SIGRH 17
01601526 18
, 19
de 20
Assessor 21
, 22
da 23
Comissão 24
Permanente 25
de 26
Disciplina 27
, 28
da 29
Secretaria 30
de 31
Estado 32
de 33
Desenvolvimento 34
Econômico 35
do 36
Distrito 37
Federal 38
. 39
155
['EXONERAR', 'DIOGO', 'DE', 'AMORIM', 'BARROS', ',', 'Farmacêutico', 'Bioquímico', ',', 'matrícula', '1672076-8', ',', 'do', 'Cargo', 'Público', 'em', 'Comissão', ',', 'Símbolo', 'CPC-08', ',', 'SIGRH', '55004987', ',', 'de', 'Gerente', ',', 'da', 'Gerência', 'de', 'Armazenamento', 'e', 'Distribuição', 'de', 'Medicamentos', 'e', 'Materiais', 'Laboratoriais', ',', 'da', 'Diretoria', 'de', 'Logística', ',', 'da', 'Subsecretaria', 'de', 'Logística', 'em', 'Saúde', ',', 'da', 'Secretaria', 'de', 'Estado', 'de', 'Saúde', 'do', 'Distrito', 'Federal', '.']
EXONERAR 0
DIOGO 1
DE 2
AMORIM 3
BARROS 4
, 5
Farmacêutico 6
Bioquímico 7
, 8
matrícula 9
1672076-8 10
, 11
do 12
Cargo 13
Público 14
em 15
Comissão 16
, 17
Símbolo 18
CPC-08 19
, 20
SIGRH 21
55004987 22
, 23
de 24
Gerent

`` 69
... 70
o 71
período 72
de 73
27/09/2008 74
a 75
08/04/2017 76
'' 77
. 78
227
['CONCEDER', 'APOSENTADORIA', 'a', 'HELENA', 'FARIAS', 'DE', 'ARAÚJO', ',', 'matrícula', '41.795-5', ',', 'no', 'Cargo', 'de', 'Agente', 'de', 'Gestão', 'Educacional/Conservação', 'e', 'Limpeza', ',', 'Nível', '11', ',', 'Padrão', '1', ',', 'Etapa', 'III', ',', 'do', 'Quadro', 'de', 'Pessoal', 'do', 'Distrito', 'Federal', ',', 'nos', 'termos', 'do', 'artigo', '3º', ',', 'incisos', 'I', ',', 'II', 'e', 'III', ',', 'e', 'parágrafo', 'único', 'da', 'Emenda', 'Constitucional', 'n°', '47', 'de', '06', 'de', 'julho', 'de', '2005', '.', 'Processo', '0008000191202/2019-81', '.']
CONCEDER 0
APOSENTADORIA 1
a 2
HELENA 3
FARIAS 4
DE 5
ARAÚJO 6
, 7
matrícula 8
41.795-5 9
, 10
no 11
Cargo 12
de 13
Agente 14
de 15
Gestão 16
Educacional/Conservação 17
e 18
Limpeza 19
, 20
Nível 21
11 22
, 23
Padrão 24
1 25
, 26
Etapa 27
III 28
, 29
do 30
Quadro 31
de 32
Pessoal 33
do 34
Distrito 35
Federal 36
, 37
nos 38
termos 39
do 4

In [14]:
len(dados_treino)

203

In [16]:
dados_teste = []

# itera sobre array de treino
for valor in np.nditer(X_test):
    # instancia variavel para mapear instancia na lista at_tokenizados
    index_ato = int(valor)
    print(index_ato)
    # instancia variavel para iterar sobre a lista de palavras
    index_token = 0
    ato = at_tokenizados[index_ato]
    print(ato)
    # itera sobre lista de palavras
    for palavra in ato:
        print(palavra, index_token)
        # é o token inicial?
        if index_token == 0:
            palavra = palavra + ' S-'+ infos[0][index_ato]
            ato[index_token] = palavra
        # é o segundo token?
        elif index_token == 1:
            palavra = palavra + ' I-'+ infos[0][index_ato]
            ato[index_token] = palavra
        # é o token final?
        elif index_token == len(at_tokenizados[index_ato])-1:
            # Já temos a informação de classe no token de início.
            palavra = palavra + ' E'
            ato[index_token] = palavra
        else:
            palavra = palavra + ' O'
            ato[index_token] = palavra
        index_token = index_token + 1
    # salva lista de palavras na lista de treino ou teste
    dados_teste.append(ato)

264
['DESIGNAR', 'VITOR', 'EMÍLIO', 'BARROS', 'DE', 'BRITO', ',', 'matrícula', 'nº', '179.182-6', ',', 'ocupante', 'do', 'cargo', 'de', 'Especialista', 'Socioeducativo', '-', 'Educador', 'Social', ',', 'para', 'substituir', 'o', 'cargo', 'de', 'Gerente', ',', 'da', 'Gerência', 'da', 'Semiliberdade', 'de', 'Taguatinga', 'I', ',', 'da', 'Diretoria', 'de', 'Semiliberdade', ',', 'da', 'Coordenação', 'de', 'Semiliberdade', 'e', 'de', 'Meio', 'Aberto', ',', 'da', 'Subsecretaria', 'do', 'Sistema', 'Socioeducativo', ',', 'da', 'Secretaria', 'de', 'Estado', 'de', 'Justiça', 'e', 'Cidadania', 'do', 'Distrito', 'Federal', ',', 'em', 'seus', 'impedimentos', 'ou', 'afastamentos', 'legais', ',', 'a', 'contar', 'de', '05/08/2020', '.']
DESIGNAR 0
VITOR 1
EMÍLIO 2
BARROS 3
DE 4
BRITO 5
, 6
matrícula 7
nº 8
179.182-6 9
, 10
ocupante 11
do 12
cargo 13
de 14
Especialista 15
Socioeducativo 16
- 17
Educador 18
Social 19
, 20
para 21
substituir 22
o 23
cargo 24
de 25
Gerente 26
, 27
da 28
Gerência 29
da 30


ato 19
que 20
retificou 21
a 22
Ordem 23
de 24
Serviço 25
nº 26
237 27
, 28
de 29
26 30
de 31
agosto 32
de 33
2016 34
, 35
publicada 36
no 37
DODF 38
nº 39
164 40
, 41
de 42
30 43
de 44
agosto 45
de 46
2016 47
, 48
página 49
22 50
, 51
o 52
ato 53
que 54
concedeu 55
o 56
5º 57
Quinquênio 58
de 59
LicençaPrêmio 60
por 61
Assiduidade 62
ao 63
servidor 64
CONSTANTINO 65
DE 66
CASTRO 67
SERRANO 68
, 69
matrícula 70
nº 71
102.374-8 72
. 73
233
['CONCEDER', 'APOSENTADORIA', 'a', 'MARIA', 'DO', 'ROSÁRIO', 'DE', 'FÁTIMA', 'SOUSA', 'OLIVEIRA', ',', 'matrícula', '40.639-2', ',', 'no', 'Cargo', 'de', 'Agente', 'de', 'Gestão', 'Educacional/Conservação', 'e', 'Limpeza', ',', 'Nível', '11', ',', 'Padrão', '1', ',', 'Etapa', 'IV', ',', 'do', 'Quadro', 'de', 'Pessoal', 'do', 'Distrito', 'Federal', ',', 'nos', 'termos', 'do', 'artigo', '3º', ',', 'incisos', 'I', ',', 'II', 'e', 'III', ',', 'e', 'parágrafo', 'único', 'da', 'Emenda', 'Constitucional', 'n°', '47', 'de', '06', 'de', 'julho', 'de', '2005', 

Educação 15
Básica 16
, 17
Padrão 18
25 19
, 20
Etapa 21
VI 22
, 23
do 24
Quadro 25
de 26
Pessoal 27
do 28
Distrito 29
Federal 30
, 31
nos 32
termos 33
do 34
artigo 35
3º 36
, 37
incisos 38
I 39
, 40
II 41
e 42
III 43
, 44
e 45
parágrafo 46
único 47
da 48
Emenda 49
Constitucional 50
n° 51
47 52
de 53
06 54
de 55
julho 56
de 57
2005 58
, 59
com 60
as 61
vantagens 62
previstas 63
no 64
artigo 65
5º 66
da 67
Lei 68
nº 69
4.584 70
, 71
de 72
08 73
de 74
julho 75
de 76
2011 77
. 78
Processo 79
00080-00035401/2020-44 80
. 81
160
['NOMEAR', 'MARIA', 'DE', 'FATIMA', 'SANTOS', 'SILVA', 'para', 'exercer', 'o', 'Cargo', 'em', 'Comissão', ',', 'Símbolo', 'CC-08', ',', 'SIGRH', '55005842', ',', 'de', 'Assessor', ',', 'da', 'Assessoria', 'de', 'Gestão', 'Participativa', 'e', 'Relações', 'Institucionais', ',', 'do', 'Gabinete', ',', 'da', 'Secretaria', 'de', 'Estado', 'de', 'Saúde', 'do', 'Distrito', 'Federal', '.']
NOMEAR 0
MARIA 1
DE 2
FATIMA 3
SANTOS 4
SILVA 5
para 6
exercer 7
o 8
Cargo 9
em 10
Co

outro 7
cargo 8
, 9
ELIZABETH 10
LIMA 11
DOS 12
SANTOS 13
FARKAS 14
do 15
Cargo 16
em 17
Comissão 18
, 19
Símbolo 20
CC-04 21
, 22
código 23
SIGRH 24
07200173 25
, 26
de 27
Assessor 28
Técnico 29
, 30
da 31
Coordenação 32
de 33
Administração 34
Geral 35
, 36
da 37
Administração 38
Regional 39
do 40
Plano 41
Piloto 42
do 43
Distrito 44
Federal 45
. 46
215
['Na', 'Ordem', 'de', 'Serviço', 'de', '13', 'de', 'novembro', 'de', '2015', 'publicada', 'no', 'DODF', 'nº', '222', 'de', '19/11/2015', ',', 'página', '27', ',', 'o', 'ato', 'que', 'concedeu', 'licença', 'prêmio', 'do', 'servidor', ':', 'CRISTIANE', 'DA', 'SILVA', 'LOURENCOO', ',', 'matrícula', ':', '0183651X', ',', 'processo', '279.001125/2015', ',', 'ONDE', 'SE', 'LÊ', ':', '“', '1°', 'quinquênio', ':', '“', '27/11/2009', 'à', '21/12/2014', '”', ',', 'LEIA-SE', ':', '“', '27/11/2009', 'à', '25/11/2014', '”', '.']
Na 0
Ordem 1
de 2
Serviço 3
de 4
13 5
de 6
novembro 7
de 8
2015 9
publicada 10
no 11
DODF 12
nº 13
222 14
de 15
19/11/201

06 59
de 60
julho 61
de 62
2005 63
. 64
Processo 65
00080-00078736/2020-57 66
. 67
181
['NOMEAR', 'BIANCA', 'SIMÕES', 'BENTLEY', ',', 'matrícula', '274.198-9', ',', 'para', 'exercer', 'o', 'Cargo', 'de', 'Natureza', 'Especial', ',', 'Símbolo', 'CNE-04', ',', 'SIGRH', '00401476', ',', 'de', 'Assessor', 'Especial', ',', 'da', 'Subsecretaria', 'de', 'Parcelamentos', 'e', 'Regularização', 'Fundiária', ',', 'da', 'Secretaria', 'de', 'Estado', 'de', 'Desenvolvimento', 'Urbano', 'e', 'Habitação', 'do', 'Distrito', 'Federal', '.']
NOMEAR 0
BIANCA 1
SIMÕES 2
BENTLEY 3
, 4
matrícula 5
274.198-9 6
, 7
para 8
exercer 9
o 10
Cargo 11
de 12
Natureza 13
Especial 14
, 15
Símbolo 16
CNE-04 17
, 18
SIGRH 19
00401476 20
, 21
de 22
Assessor 23
Especial 24
, 25
da 26
Subsecretaria 27
de 28
Parcelamentos 29
e 30
Regularização 31
Fundiária 32
, 33
da 34
Secretaria 35
de 36
Estado 37
de 38
Desenvolvimento 39
Urbano 40
e 41
Habitação 42
do 43
Distrito 44
Federal 45
. 46
35
['EXONERAR', ',', 'a', 'pedido', ',',

, 72
de 73
16 74
de 75
dezembro 76
de 77
1998 78
, 79
e 80
artigo 81
2º 82
da 83
Emenda 84
Constitucional 85
nº 86
47 87
, 88
de 89
06 90
de 91
julho 92
de 93
2005 94
. 95
Processo 96
00080-00030744/2020-12 97
. 98
111
['CONCEDER', 'APOSENTADORIA', 'a', 'MARIA', 'MEIRE', 'SOUSA', 'ÁVILA', ',', 'matrícula', '40.965-0', ',', 'no', 'Cargo', 'de', 'Agente', 'de', 'Gestão', 'Educacional/Conservação', 'e', 'Limpeza', ',', 'Nível', '11', ',', 'Padrão', '1', ',', 'Etapa', 'IV', ',', 'do', 'Quadro', 'de', 'Pessoal', 'do', 'Distrito', 'Federal', ',', 'nos', 'termos', 'do', 'Artigo', '6º', ',', 'incisos', 'I', ',', 'II', ',', 'III', 'e', 'IV', ',', 'da', 'Emenda', 'Constitucional', 'nº', '41', ',', 'de', '31', 'de', 'dezembro', 'de', '2003', 'e', 'artigo', '2º', 'da', 'Emenda', 'Constitucional', 'nº', '47', ',', 'de', '06', 'de', 'julho', 'de', '2005', '.', 'Processo', '00080-00030396/2020-83', '.']
CONCEDER 0
APOSENTADORIA 1
a 2
MARIA 3
MEIRE 4
SOUSA 5
ÁVILA 6
, 7
matrícula 8
40.965-0 9
, 10
no 

NOTA 5
DE 6
EMPENHO 7
Nº 8
2020NE05466 9
Processo 10
: 11
00060-00265084/2020-63 12
PUBLICADO 13
NA 14
PÁGN 15
80 16
DO 17
DODF 18
Nº 19
123 20
, 21
QUINTA-FEIRA 22
, 23
02 24
DE 25
JULHO 26
DE 27
2020 28
, 29
COM 30
O 31
SEGUINTE 32
TEXTO 33
: 34
´´EXTRATO 35
DE 36
NOTA 37
DE 38
EMPENHO 39
Nº 40
2020NE05466 41
Processo 42
: 43
00060-00265084/2020-63 44
. 45
Partes 46
: 47
DISTRITO 48
FEDERAL 49
, 50
por 51
intermédio 52
de 53
sua 54
SECRETARIA 55
DE 56
ESTADO 57
DE 58
SAÚDE 59
DO 60
DISTRITO 61
FEDERAL 62
e 63
a 64
empresa 65
MERCANTIL 66
BARRETO 67
COMERCIAL 68
DE 69
PRODUTOS 70
HOSPITALARES 71
LTDA 72
. 73
CNPJ 74
Nº 75
15.031.173/0001-44 76
. 77
OBJETO 78
: 79
AQUISIÇÃO 80
DE 81
AMICACINA 82
( 83
SULFATO 84
) 85
SOLUCAO 86
INJETAVEL 87
250 88
MG/ML 89
AMPOLA 90
2 91
ML 92
, 93
conforme 94
Ata 95
de 96
Registro 97
de 98
Preço 99
nº 100
068/2020-A 101
SES/DF 102
e 103
Pedido 104
de 105
Aquisição 106
de 107
Material 108
nº 109
5-20/PAM002605 110
e 111
Autorização 112
de 113
Fornecimen

47 90
, 91
de 92
06 93
de 94
julho 95
de 96
2005 97
. 98
Processo 99
0008000002565/2020-95 100
. 101
79
['EXONERAR', ',', 'a', 'pedido', ',', 'MARCOS', 'PAULO', 'THOME', ',', 'matrícula', '1694552-2', ',', 'Analista', 'de', 'Atividades', 'do', 'Hemocentro', ',', 'especialidade', 'Analista', 'de', 'Laboratório', ',', 'a', 'contar', 'de', '03/08/202', ',', 'nos', 'termos', 'do', 'artigo', '51', 'da', 'Lei', 'Complementar', '840/2011', ',', 'conforme', 'processo', 'SEI', 'nº', '00063-00003412/2020-55', '.']
EXONERAR 0
, 1
a 2
pedido 3
, 4
MARCOS 5
PAULO 6
THOME 7
, 8
matrícula 9
1694552-2 10
, 11
Analista 12
de 13
Atividades 14
do 15
Hemocentro 16
, 17
especialidade 18
Analista 19
de 20
Laboratório 21
, 22
a 23
contar 24
de 25
03/08/202 26
, 27
nos 28
termos 29
do 30
artigo 31
51 32
da 33
Lei 34
Complementar 35
840/2011 36
, 37
conforme 38
processo 39
SEI 40
nº 41
00063-00003412/2020-55 42
. 43
126
['TORNAR', 'SEM', 'EFEITO', 'no', 'DODF', 'nº', '179', ',', 'de', '19', 'de', 'setembro', '

In [17]:
len(dados_teste)

68

### Dump de listas para arquivos txt

In [56]:
# treino
with open('treino.txt', 'a') as f:
    for ato in dados_treino:
        for palavra in ato:
            f.write(palavra + '\n')

# teste
with open('teste.txt', 'a') as f:
    for ato in dados_teste:
        for palavra in ato:
            f.write(palavra + '\n')